# ARVTON - Virtual Try-On Fine-Tuning

This notebook fine-tunes the ARVTON GAN refinement model on Google Colab's free T4 GPU.

**Steps:**
1. Clone repo & install deps (~3 min)
2. Download/prepare dataset (~5 min)
3. Train GAN model (~30-60 min for 50 epochs)
4. Download checkpoints to local machine

---
**GPU requirement:** T4 (free tier) or A100 (Pro)

## 1. Setup Environment (~3 min)

In [ ]:
# Check GPU availability
!nvidia-smi
import torch
print(f"\nPyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
# Clone the ARVTON repository
# Replace with your actual repo URL
REPO_URL = "https://github.com/Gandharv2323/AR-PROTOTYPE-4.git"  # <-- UPDATE THIS

import os
if not os.path.exists("/content/AR-PROTOTYPE-4"):
    !git clone {REPO_URL} /content/AR-PROTOTYPE-4
    print("[OK] Repo cloned")
else:
    !cd /content/AR-PROTOTYPE-4 && git pull
    print("[OK] Repo updated")

%cd /content/AR-PROTOTYPE-4

In [ ]:
# Install dependencies (~2 min)
!pip install -q -r requirements.txt
!pip install -q peft accelerate
print("[OK] Dependencies installed")

In [ ]:
# Setup directories
!python setup_local.py

## 2. Mount Google Drive (for saving checkpoints)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create checkpoint directory in Drive
DRIVE_CKPT = "/content/drive/MyDrive/ARVTON/checkpoints"
os.makedirs(DRIVE_CKPT, exist_ok=True)
print(f"[OK] Checkpoints will save to: {DRIVE_CKPT}")

## 3. Prepare Dataset

**Option A:** Upload your own dataset (person + garment image pairs)  
**Option B:** Generate synthetic dataset using the pipeline

In [ ]:
# Option A: Upload your own dataset
# Upload a zip file containing:
#   - person_001.jpg, person_002.jpg, ...
#   - garment_001.jpg, garment_002.jpg, ...
#   - mask_001.png, mask_002.png, ...
#   - densepose_001.png, densepose_002.png, ...

# Uncomment below to upload from your computer:
# from google.colab import files
# uploaded = files.upload()  # Upload your dataset.zip
# !unzip -q dataset.zip -d data/arvton/datasets/custom/

print("Skip this cell if using Option B (synthetic data)")

In [ ]:
# Option B: Generate synthetic dataset (~5-10 min)
# This uses Stable Diffusion to create person + garment pairs

GENERATE_SYNTHETIC = True  # Set to False if you uploaded your own data
NUM_SAMPLES = 100  # Number of training pairs to generate (more = better quality)

if GENERATE_SYNTHETIC:
    print(f"Generating {NUM_SAMPLES} synthetic training pairs...")
    print(f"Estimated time: ~{NUM_SAMPLES * 3}s ({NUM_SAMPLES * 3 / 60:.0f} min)")
    !python -m pipeline.synthetic_gen --target-count {NUM_SAMPLES}
    print("[OK] Synthetic dataset generated")

In [ ]:
# Verify dataset
import json
from pathlib import Path

manifest = Path("data/arvton/datasets/train_manifest.json")
if manifest.exists():
    with open(manifest) as f:
        data = json.load(f)
    if isinstance(data, dict) and "entries" in data:
        count = len(data["entries"])
    elif isinstance(data, list):
        count = len(data)
    else:
        count = 0
    print(f"[OK] Training manifest: {count} entries")
else:
    print("[WARN] No manifest found. Upload data or run synthetic generation first.")

## 4. Configure Training

In [ ]:
# Training hyperparameters
# Optimized for Colab T4 (16 GB VRAM)

EPOCHS = 50           # Total training epochs
BATCH_SIZE = 2        # T4: 2, A100: 8
LEARNING_RATE = 1e-4  # AdamW learning rate
USE_AMP = True        # Mixed precision (saves VRAM)
USE_LORA = True       # LoRA adapters (saves VRAM)
LORA_RANK = 16        # LoRA rank (higher = more capacity, more VRAM)
IMAGE_SIZE = (512, 512)  # T4: 512x512, A100: 768x1024

print(f"Training config:")
print(f"  Epochs:     {EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  LR:         {LEARNING_RATE}")
print(f"  AMP:        {USE_AMP}")
print(f"  LoRA:       {USE_LORA} (rank={LORA_RANK})")
print(f"  Image size: {IMAGE_SIZE}")
print(f"")
print(f"Estimated training time: ~{EPOCHS * 0.8:.0f} min on T4")
print(f"Estimated VRAM usage:    ~6-8 GB")

## 5. Train! (~30-60 min)

In [ ]:
import time
start_time = time.time()

# Build the training command
cmd = f"python -m pipeline.train_local --epochs {EPOCHS} --batch-size {BATCH_SIZE} --lr {LEARNING_RATE}"
if USE_AMP:
    cmd += " --amp"
if USE_LORA:
    cmd += f" --lora --lora-rank {LORA_RANK}"

print(f"Running: {cmd}")
print(f"Estimated time: ~{EPOCHS * 0.8:.0f} min")
print("=" * 60)

!{cmd}

elapsed = (time.time() - start_time) / 60
print(f"\n{'=' * 60}")
print(f"Training complete in {elapsed:.1f} minutes")

## 6. Save Checkpoints to Google Drive

In [ ]:
import shutil

# Copy checkpoints to Drive for persistence
local_ckpts = Path("data/arvton/checkpoints")
if local_ckpts.exists():
    for ckpt in local_ckpts.rglob("*.pt"):
        dest = Path(DRIVE_CKPT) / ckpt.name
        shutil.copy2(str(ckpt), str(dest))
        print(f"  Saved: {dest.name} ({ckpt.stat().st_size / 1024**2:.1f} MB)")
    print(f"\n[OK] Checkpoints saved to Google Drive: {DRIVE_CKPT}")
else:
    print("[WARN] No checkpoints found")

## 7. Evaluate Model Quality

In [ ]:
# Run inference on a sample to check quality
from PIL import Image
import matplotlib.pyplot as plt

# Find sample images
sample_dir = Path("data/arvton/datasets")
persons = sorted(sample_dir.rglob("person_*.jpg"))[:3]
garments = sorted(sample_dir.rglob("garment_*.jpg"))[:3]

if persons and garments:
    fig, axes = plt.subplots(len(persons), 3, figsize=(15, 5 * len(persons)))
    if len(persons) == 1:
        axes = axes.reshape(1, -1)
    
    for i, (p, g) in enumerate(zip(persons, garments)):
        axes[i, 0].imshow(Image.open(p))
        axes[i, 0].set_title("Person")
        axes[i, 0].axis("off")
        
        axes[i, 1].imshow(Image.open(g))
        axes[i, 1].set_title("Garment")
        axes[i, 1].axis("off")
        
        # Check if a result exists
        result_path = Path(f"data/arvton/outputs/samples/result_{i:03d}.jpg")
        if result_path.exists():
            axes[i, 2].imshow(Image.open(result_path))
            axes[i, 2].set_title("Try-On Result")
        else:
            axes[i, 2].text(0.5, 0.5, "Run inference\nto see result", 
                           ha='center', va='center', fontsize=14)
        axes[i, 2].axis("off")
    
    plt.tight_layout()
    plt.savefig("data/arvton/outputs/training_samples.png", dpi=100)
    plt.show()
    print("[OK] Sample visualization saved")
else:
    print("[WARN] No sample images found for visualization")

## 8. Download Checkpoint to Local Machine

After training, download the best checkpoint to use locally.

On your local machine:
1. Copy the `.pt` file to `data/arvton/checkpoints/gan/`
2. Run: `python run_local.py`
3. The backend will automatically use the fine-tuned weights

In [ ]:
# Download the latest checkpoint
from google.colab import files

ckpts = sorted(Path(DRIVE_CKPT).glob("*.pt"))
if ckpts:
    latest = ckpts[-1]
    size_mb = latest.stat().st_size / 1024**2
    print(f"Downloading: {latest.name} ({size_mb:.1f} MB)")
    print(f"Estimated download time: ~{size_mb / 5:.0f}s")
    files.download(str(latest))
else:
    print("[WARN] No checkpoints to download. Train first!")

---
## Summary

After fine-tuning:
1. Download the `.pt` checkpoint file
2. Place it in your local project at `data/arvton/checkpoints/gan/`
3. Start the local backend: `python run_local.py`
4. Test with Flutter app: `cd arvton_app && flutter run -d chrome`
5. When satisfied, deploy to cloud (Docker + AWS/GCP)